### **DESARROLLO Y PRUEBA DE LOS ENDPOINTS**
#### Se desarrolla cada endpoint de manera aislada, probando sus respuestas y asegurando su efectividad antes de integrarlos en el archivo `main.py`.

#### Se utiliza el dataset procesado previamente.

In [12]:
import pandas as pd

In [14]:
# Cargamos el dataset que se procesó previamente (parquet)
movies_df = pd.read_parquet('C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\Dataset\dataset_final.parquet', engine= 'fastparquet')

- #### **Cantidad de filmaciones por mes:**

In [18]:
def cantidad_filmaciones_mes(mes: str) -> dict:
    """
    Devuelve la cantidad de películas que fueron estrenadas en el mes consultado.
    
    Parámetros:
    mes (str): Nombre del mes en español.
    
    Retorno:
    dict: Mensaje con la cantidad de películas estrenadas en el mes.
    """
    # Diccionario de meses en español a números
    meses_diccionario = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    
    # Convertimos el mes a su número correspondiente
    mes_numero = meses_diccionario.get(mes.lower())
    
    if mes_numero:
        # Filtramos las películas del mes del 'release_date'
        cantidad = movies_df[movies_df['release_date'].dt.month == mes_numero].shape[0]
        return {"mensaje": f"{cantidad} películas fueron estrenadas en el mes de {mes.capitalize()}"}
    else:
        return {"error": "Mes no válido. Por favor, ingrese en español un mes válido."}


In [19]:
cantidad_filmaciones_mes("febrero")

{'mensaje': '27122 películas fueron estrenadas en el mes de Febrero'}

- #### **Cantidad de filmaciones por día:**

In [20]:
def cantidad_filmaciones_dia(dia: str) -> dict:
    """
    Devuelve la cantidad de películas que fueron estrenadas en el día de la semana consultado.
    
    Parámetros:
    - dia (str): Nombre del día en español.
    
    Retorno:
    - dict: Mensaje con la cantidad de películas estrenadas en el día.
    """
    # Diccionario de días de la semana en español a su número correspondiente (lunes=0, domingo=6)
    dias_diccionario = {
        'lunes': 0, 'martes': 1, 'miércoles': 2, 'jueves': 3, 
        'viernes': 4, 'sábado': 5, 'domingo': 6
    }
    
    # Convertimos el día a su número correspondiente
    dia_numero = dias_diccionario.get(dia.lower())
    
    if dia_numero is not None:
        # Filtramos las películas por el día de la semana del 'release_date'
        cantidad = movies_df[movies_df['release_date'].dt.dayofweek == dia_numero].shape[0]
        return {"mensaje": f"{cantidad} películas fueron estrenadas el día {dia.capitalize()}"}
    else:
        return {"error": "Por favor, ingrese en español un día válido."}

In [21]:
cantidad_filmaciones_dia('jueves')

{'mensaje': '71356 películas fueron estrenadas el día Jueves'}

- #### **Por el título de la película y score:**

In [4]:
def score_titulo(titulo: str) -> dict:
    """
    Devuelve el título, año de estreno y score de una filmación.

    Parámetros:
    - titulo (str): Título de la filmación.

    Retorno:
    - dict: Mensaje con el título, año de estreno y el score de la película.
    """
    # Filtramos la película por el título
    film = movies_df[movies_df['title'].str.lower() == titulo.lower()]
    
    if not film.empty:
        # Extraemos el año de estreno y el score
        titulo_film = film['title'].values[0]
        año_estreno = film['release_year'].values[0]
        popularity = film['popularity'].values[0]
        
        return {
            "mensaje": f"La película '{titulo_film}' fue estrenada en el año {año_estreno} con un score de {popularity}"
        }
    else:
        return {"error": f"No se encontró la película con el título '{titulo}'."}


In [5]:
score_titulo('iron man')

{'mensaje': "La película 'Iron Man' fue estrenada en el año 2008 con un score de 22.073099"}

- #### **Por el título de la película y valoraciones:**

In [38]:
def votos_titulo(titulo: str) -> dict:
    """
    Devuelve la cantidad de votos y el promedio de votaciones de una película.
    
    Parámetros:
    - titulo (str): Título de la película.

    Retorno:
    - dict: Mensaje con la cantidad de votos y el promedio, o un mensaje si no cumple el mínimo de votos.
    """
    # Filtramos la película por el título
    film = movies_df[movies_df['title'].str.lower() == titulo.lower()]
    
    if not film.empty:
        votos = film['vote_count'].values[0]
        promedio_votos = film['vote_average'].values[0]
        
        # Verificamos si la película tiene al menos 2000 votos
        if votos >= 2000:
            return {
                "mensaje": f"La película '{titulo}' tiene {votos} valoraciones, con un promedio de {promedio_votos}"
            }
        else:
            return {
                "mensaje": f"La película '{titulo}' no cumple con el mínimo de 2000 votos. Solo tiene {votos} votos."
            }
    else:
        return {"error": f"No se encontró la película con el título '{titulo}'."}


In [39]:
votos_titulo('iron man')

{'mensaje': "La película 'iron man' tiene 8951.0 valoraciones, con un promedio de 7.4"}

- #### **Por el nombre del actor:**

In [7]:
def get_actor(nombre_actor: str) -> dict:
    """
    Devuelve el éxito de un actor medido a través del retorno total y promedio,
    así como la cantidad de películas en las que ha participado.

    Parámetros:
    - nombre_actor (str): Nombre del actor a consultar.

    Retorno:
    - dict: Diccionario con la cantidad de películas en las que ha participado,
            el retorno total y el retorno promedio por filmación.
            Si no se encuentra el actor, devuelve un mensaje de error.g
    """
    # Filtramos por el nombre del actor
    actor_data = movies_df[movies_df['actor_name'].str.contains(nombre_actor, case=False, na=False)]
    
    # Verificamos si se encontraron resultados
    if not actor_data.empty:
        cantidad_filmaciones = actor_data.shape[0]
        total_retorno = actor_data['return'].sum()
        promedio_retorno = total_retorno / cantidad_filmaciones
        
        # Retorno exitoso con los datos del actor
        return {
            "actor": nombre_actor,
            "cantidad_filmaciones": cantidad_filmaciones,
            "total_retorno": total_retorno,
            "promedio_retorno": promedio_retorno
        }
    else:
        # Retorno en caso de no encontrar al actor
        return {"error": f"No se encontraron películas para el actor {nombre_actor}"}


In [8]:
get_actor('Ben Affleck')

{'actor': 'Ben Affleck',
 'cantidad_filmaciones': 595,
 'total_retorno': np.float64(1973.6386124328706),
 'promedio_retorno': np.float64(3.317039684761127)}

- #### **Sistema de Recomendación con Similitud del Coseno**


In [2]:
import pandas as pd
from scipy import sparse
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# Dataset
model_df = pd.read_parquet('C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\ModelML\model_ml.parquet')

# Matriz TF-IDF
tfidf_matrix = sparse.load_npz(r'C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\ModelML\tfidf_matrix.npz')

# Vectorizador
with open(r'C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\ModelML\tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

# Matriz de Similitud    
cosine_sim = np.load('C:\DYNAMO\DATA SCIENCE\Proyectos\Proyecto-MLOps\ModelML\cosine_sim.npy')   

In [13]:
def recomendacion(titulo: str, num_recomendaciones: int = 5) -> list:
    """
    Dada una película, devuelve una lista con los títulos de las películas más similares.

    Parámetros:
    - titulo (str): El título de la película para la cual se quieren recomendaciones.
    - num_recomendaciones (int): Número de recomendaciones a devolver. Por defecto es 5.

    Retorna:
    - list: Lista con los títulos de las películas recomendadas.
    """
    
    # Se verifica si está disponible la película
    if titulo not in model_df['title'].values:
        return ["El título no está en el dataset."]
    
    # Se obtiene el índice de la película 
    idx = model_df[model_df['title'] == titulo].index[0]
    
    # Se hace el calculo del coseno entre la película y todas las demás
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Se ordenan las películas en base a la similitud (de mayor a menor)
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Se obtiene los índices de las películas más similares, excluyendo la película original
    sim_scores = sim_scores[1:num_recomendaciones + 1]
    
    # Se obtiene las películas más similares
    movie_indices = [i[0] for i in sim_scores]
    
    return model_df['title'].iloc[movie_indices].tolist()

In [14]:
print(recomendacion('Spider-Man', num_recomendaciones=5))


['The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Spider-Man 2', 'Kick-Ass', 'Austin High']


- #### **Sistema de Recomendación con K-vecinos**

In [1]:
from sklearn.neighbors import NearestNeighbors

In [8]:
# Se entrena el modelo KNN con la matriz TF-IDF
knn_model = NearestNeighbors(metric='cosine', algorithm='brute')
knn_model.fit(tfidf_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [10]:
def recomendacion_Knn(titulo: str, num_recomendaciones: int = 5):
    """
  Devuelve una lista de películas similares a la película proporcionada usando K-Vecinos.
    
    Parámetros:
    - titulo (str): El título de la película para la cual se quieren obtener recomendaciones.
    - num_recomendaciones (int, opcional): Número de películas a recomendar (por defecto 5).

    Retorna:
    - dict: Diccionario con la lista de títulos de películas recomendadas o un mensaje de error si el título no se encuentra.

    Descripción:
    La función utiliza el algoritmo K-Nearest Neighbors para encontrar las películas más similares
    a la película solicitada en base a la similitud de coseno entre las descripciones.
    """
    # Verificar si el título proporcionado está en el dataset
    if titulo not in model_df['title'].values:
        return {"error": "El título no está en el dataset."}
    
    # Obtener el índice de la película solicitada
    idx = model_df[model_df['title'] == titulo].index[0]
    
    # Usar el modelo KNN para encontrar los K vecinos más cercanos
    distances, indices = knn_model.kneighbors(tfidf_matrix[idx], n_neighbors=num_recomendaciones + 1)
    
    # Obtener los títulos de las películas recomendadas (excluyendo la película original)
    movie_indices = indices.flatten()[1:num_recomendaciones + 1]
    recomendaciones = model_df['title'].iloc[movie_indices].tolist()
    
    return {"recomendaciones": recomendaciones}

In [11]:
print(recomendacion_Knn('Spider-Man', num_recomendaciones=5))

{'recomendaciones': ['The Amazing Spider-Man', 'The Amazing Spider-Man 2', 'Spider-Man 2', 'Kick-Ass', 'Austin High']}
